<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Judy/European_Call_FiniteDifference_Jax_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   6147      0 --:--:-- --:--:-- --:--:--  6124
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 19 kB/s 
     |████████████████████████████████| 1.0 MB 40.3 MB/s 
+ set +ex
Installation succeeded!


In [2]:
nstock = 3

# Finite Difference Calculate Delta

In [11]:
################################# TEST ########################################
#%%writefile cupy_dataset.py
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps,))
    dt = jnp.array(T[0]/numsteps)
    def time_step(t, val):
        dx =  drift * dt * val[t-1,:] + val[t-1,:] * jnp.sqrt(dt) * noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] + dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)


class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, seed=15, stocks=3):  # 3 stocks
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = 1
        self.seed = seed
        np.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num >= self.max_length:
            raise StopIteration
        # D = cupy.zeros((self.N_BATCH, self.N_STOCKS), dtype=cupy.float32)
        # Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)
        Y = cupy.zeros((self.N_BATCH, self.N_STOCKS + 1), dtype=cupy.float32)
        X = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)

        for op in range(self.N_BATCH):
          
          key = random.PRNGKey(self.seed)
          initial_stocks = jnp.array(np.random.random(self.N_STOCKS) * 200)
          # cov=np.random.random((3,3))
          # cov=np.matmul(cov,cov.T)
          corr = jnp.diag(jnp.array([1]*self.N_STOCKS)) # assume no correlation between stocks here
          sigma = jnp.array(np.random.random(self.N_STOCKS) * 0.4)
          cov = (jnp.diag(sigma)).dot(corr).dot(jnp.diag(sigma))
          r = jnp.repeat(jnp.array(np.random.random(1) * 0.1), self.N_STOCKS)
          drift = r # To match BS, use drift = r
          T = jnp.array([self.T] * self.N_STOCKS)
          K = np.random.random(1) * 200

          def ftest(INITAL_STOCKS):
            fast_simple = jax.jit(Brownian_motion, static_argnums=2)
            fast_simple(key, INITAL_STOCKS, self.N_STEPS, drift, cov, T)

            numsamples = self.N_PATHS # num of paths
            keys = jax.random.split(key, numsamples)

            batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
            out = batch_simple(keys, INITAL_STOCKS, self.N_STEPS, drift, cov, T)

            European_Call_price = np.mean(np.maximum((np.mean(out[:,self.N_STEPS,:], axis=1)) - K, 0) * jnp.exp(-r[0] * T[0]))
            return(European_Call_price)
          
          # Y[op] = ftest(initial_stocks)
          option_price = ftest(initial_stocks)
          #Finite Difference Delta
          P1 = ftest((initial_stocks + jnp.array([1,0,0])))
          P2 = ftest(initial_stocks + jnp.array([0,1,0]))
          P3 = ftest(initial_stocks + jnp.array([0,0,1]))
          D1 = P1 - Y[op]
          D2 = P2 - Y[op]
          D3 = P3 - Y[op]
          # D[op,] = cupy.array([P1 - Y[op],P2 - Y[op], P3 - Y[op]])  
          # deltas = cupy.array([P1 - option_price,P2 - option_price, P3 - option_price])
          Y[op,] = cupy.array([option_price, P1 - option_price, P2 - option_price, P3 - option_price])
          paras = (T, jnp.repeat(jnp.array(K), self.N_STOCKS), initial_stocks, sigma, drift, r)
          paras = np.column_stack(paras).reshape(1,-1)[0]
          X[op,] = cupy.array(paras)

        self.num += 1
        # return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()),from_dlpack(D.toDlpack()))
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))


ds = NumbaOptionDataSet(2, number_path = 100000, batch = 2, seed = 15, stocks=3)
for i in ds:
    print(i)

(tensor([[1.0000e+00, 6.0895e+01, 1.6976e+02, 1.4462e-01, 3.0592e-02, 3.0592e-02,
         1.0000e+00, 6.0895e+01, 3.5779e+01, 1.1016e-01, 3.0592e-02, 3.0592e-02,
         1.0000e+00, 6.0895e+01, 1.0873e+01, 2.1200e-01, 3.0592e-02, 3.0592e-02],
        [1.0000e+00, 4.2110e+01, 2.2348e+01, 1.0566e-01, 8.0708e-02, 8.0708e-02,
         1.0000e+00, 4.2110e+01, 4.9980e+01, 2.8711e-01, 8.0708e-02, 8.0708e-02,
         1.0000e+00, 4.2110e+01, 1.8353e+02, 3.4629e-01, 8.0708e-02, 8.0708e-02]],
       device='cuda:0'), tensor([[13.2316,  0.3231,  0.3198,  0.3203],
        [46.5336,  0.3334,  0.3330,  0.3338]], device='cuda:0'))
(tensor([[1.0000e+00, 9.4695e+01, 3.3449e+01, 8.0092e-02, 7.6051e-02, 7.6051e-02,
         1.0000e+00, 9.4695e+01, 9.3413e+00, 3.9942e-01, 7.6051e-02, 7.6051e-02,
         1.0000e+00, 9.4695e+01, 7.8845e+00, 1.4911e-01, 7.6051e-02, 7.6051e-02],
        [1.0000e+00, 1.0755e+02, 1.0194e+02, 1.8604e-01, 2.9900e-02, 2.9900e-02,
         1.0000e+00, 1.0755e+02, 1.8901e+02, 5.6

# Jax.Grad Calculate Delta(Not working)

In [ ]:
################################# TEST ########################################
#%%writefile cupy_dataset.py
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key, jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps,))
    dt = jnp.array(T[0]/numsteps)
    def time_step(t, val):
        dx =  drift * dt * val[t-1,:] + val[t-1,:] * jnp.sqrt(dt) * noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] + dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)


class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, seed=15, stocks=3):  # 3 stocks
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = 1
        self.seed = seed
        np.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num >= self.max_length:
            raise StopIteration
        D = cupy.zeros((self.N_BATCH, self.N_STOCKS), dtype=cupy.float32)
        Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)
        X = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)

        for op in range(self.N_BATCH):
          
          key = random.PRNGKey(self.seed)
          initial_stocks = jnp.array(np.random.random(self.N_STOCKS) * 200)
          # cov=np.random.random((3,3))
          # cov=np.matmul(cov,cov.T)
          corr = jnp.diag(jnp.array([1]*self.N_STOCKS)) # assume no correlation between stocks here
          sigma = jnp.array(np.random.random(self.N_STOCKS) * 0.4)
          cov = (jnp.diag(sigma)).dot(corr).dot(jnp.diag(sigma))
          r = jnp.repeat(jnp.array(np.random.random(1) * 0.1), self.N_STOCKS)
          drift = r # To match BS, use drift = r
          T = jnp.array([self.T] * self.N_STOCKS)
          K = np.random.random(1) * 200

          def ftest(INITAL_STOCKS):
            fast_simple = jax.jit(Brownian_motion, static_argnums=2)
            fast_simple(key, INITAL_STOCKS, self.N_STEPS, drift, cov, T)

            numsamples = self.N_PATHS # num of paths
            keys = jax.random.split(key, numsamples)

            batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
            out = batch_simple(keys, INITAL_STOCKS, self.N_STEPS, drift, cov, T)

            European_Call_price = jnp.mean(jnp.max((jnp.mean(out[:,self.N_STEPS,:], axis=1)) - K, 0) * jnp.exp(-r[0] * T[0]))
            return(European_Call_price)
          
          Y[op] = ftest(initial_stocks)
          #Jax Grad calculation (Not Working)
          delta_f = grad(ftest)
          D[op,]= cupy.array(delta_f(initial_stocks))
          paras = (T, jnp.repeat(jnp.array(K), self.N_STOCKS), initial_stocks, sigma, drift, r)
          paras = np.column_stack(paras).reshape(1,-1)[0]
          X[op,] = cupy.array(paras)

        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()),from_dlpack(D.toDlpack()))


ds = NumbaOptionDataSet(2, number_path = 100000, batch = 2, seed = 15, stocks=3)
for i in ds:
    print(i)

(tensor([[1.0000e+00, 6.0895e+01, 1.6976e+02, 1.4462e-01, 3.0592e-02, 3.0592e-02,
         1.0000e+00, 6.0895e+01, 3.5779e+01, 1.1016e-01, 3.0592e-02, 3.0592e-02,
         1.0000e+00, 6.0895e+01, 1.0873e+01, 2.1200e-01, 3.0592e-02, 3.0592e-02],
        [1.0000e+00, 4.2110e+01, 2.2348e+01, 1.0566e-01, 8.0708e-02, 8.0708e-02,
         1.0000e+00, 4.2110e+01, 4.9980e+01, 2.8711e-01, 8.0708e-02, 8.0708e-02,
         1.0000e+00, 4.2110e+01, 1.8353e+02, 3.4629e-01, 8.0708e-02, 8.0708e-02]],
       device='cuda:0'), tensor([ 58.9658, 245.0371], device='cuda:0'), tensor([[0., 0., 0.],
        [0., 0., 0.]], device='cuda:0'))
(tensor([[1.0000e+00, 9.4695e+01, 3.3449e+01, 8.0092e-02, 7.6051e-02, 7.6051e-02,
         1.0000e+00, 9.4695e+01, 9.3413e+00, 3.9942e-01, 7.6051e-02, 7.6051e-02,
         1.0000e+00, 9.4695e+01, 7.8845e+00, 1.4911e-01, 7.6051e-02, 7.6051e-02],
        [1.0000e+00, 1.0755e+02, 1.0194e+02, 1.8604e-01, 2.9900e-02, 2.9900e-02,
         1.0000e+00, 1.0755e+02, 1.8901e+02, 5.66